<a href="https://colab.research.google.com/github/Nickqq627/7112029017/blob/main/week13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import yfinance as yf
import numpy as np
import random

# 參數設定
NUM_GENERATIONS = 100   # 世代數
POPULATION_SIZE = 50    # 群體大小
MUTATION_RATE = 0.1     # 突變機率
TOURNAMENT_SIZE = 5     # 錦標賽選擇大小

# 股票代碼與時間範圍
TICKERS = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
START_DATE = "2020-01-01"
END_DATE = "2023-01-01"

# 取得股票資料
def fetch_stock_data(tickers, start, end):
    data = yf.download(tickers, start=start, end=end)["Adj Close"]
    returns = data.pct_change().dropna()
    return returns

returns = fetch_stock_data(TICKERS, START_DATE, END_DATE)
mean_returns = returns.mean()
cov_matrix = returns.cov()

# 適應函數：回報率/風險（夏普比率）
def fitness_function(weights):
    portfolio_return = np.dot(weights, mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = portfolio_return / portfolio_volatility
    return sharpe_ratio

# 初始化群體
def initialize_population(size, num_assets):
    return [np.random.randint(0, 2, num_assets) for _ in range(size)]

# 計算權重
def calculate_weights(chromosome):
    if np.sum(chromosome) == 0:
        return np.ones(len(chromosome)) / len(chromosome)
    return chromosome / np.sum(chromosome)

# 選擇：錦標賽選擇
def tournament_selection(population, fitness_scores):
    selected = random.sample(list(zip(population, fitness_scores)), TOURNAMENT_SIZE)
    return max(selected, key=lambda x: x[1])[0]

# 交配
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    child1 = np.concatenate((parent1[:point], parent2[point:]))
    child2 = np.concatenate((parent2[:point], parent1[point:]))
    return child1, child2

# 突變
def mutate(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

# 主程式
def genetic_algorithm():
    num_assets = len(TICKERS)
    population = initialize_population(POPULATION_SIZE, num_assets)

    for generation in range(NUM_GENERATIONS):
        fitness_scores = [fitness_function(calculate_weights(chromosome)) for chromosome in population]
        new_population = []

        for _ in range(POPULATION_SIZE // 2):
            parent1 = tournament_selection(population, fitness_scores)
            parent2 = tournament_selection(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1, MUTATION_RATE))
            new_population.append(mutate(child2, MUTATION_RATE))

        population = new_population

    # 找到最佳染色體
    fitness_scores = [fitness_function(calculate_weights(chromosome)) for chromosome in population]
    best_chromosome = population[np.argmax(fitness_scores)]
    best_weights = calculate_weights(best_chromosome)

    return best_chromosome, best_weights, max(fitness_scores)

best_chromosome, best_weights, best_fitness = genetic_algorithm()

# 輸出結果
print("最佳選股結果:", best_chromosome)
print("最佳權重:", best_weights)
print("最佳適應度 (夏普比率):", best_fitness)

[*********************100%***********************]  5 of 5 completed


最佳選股結果: [1 0 0 0 1]
最佳權重: [0.5 0.  0.  0.  0.5]
最佳適應度 (夏普比率): 0.06549569871123298
